In [1]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="https://openaiykus.openai.azure.com/"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [1]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",#2024-02-01
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

## 1. Understand the AOAI Models' capabilities. Start with the latest model, prove your idea , then test with smaller models. 
Model size is critical for better performance.

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

** Length ** control (specify desired output length e.g.: number of words)

** Tone ** control (e.g.: polite, passionate, professional, technical, funny, casual, serious etc.)

** Style ** control (e.g.: in the style of Shakespeare, JK Rowling, Nelson Mandela etc.)

** Audience ** control (e.g.: a 5-year-old can understand etc)

** Context ** control (e.g.: news, novel, textbook, report, white paper, blog etc.)

In [18]:
base_system_message = "You are a helpful assistant."
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = " Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on brands of The Very Group"
# Write a 2 paragraph inspiring poem about The Very Group company
# Write a 2 paragraph inspiring poem focussing on products of the Very Group company in a funny way
# Write a 2 paragraph inspiring poem focussing on products of The Very Group company in the style of Shakespeare
# Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on products of The Very Group company


In [19]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

In a world filled with cheerful toys and pretty dresses, 
There’s a place called The Very Group, that truly impresses.
It has Lego blocks to build your fantasy tower,
Barbie dolls, tea parties, you could play for an hour! 
Ted Baker clothes to make you look bright,
With Converse shoes, you’ll run as light as the night.

So dream big, little one, let your imagination take flight,
With The Very Group, every day is a colorful sight.
From Littlewoods Home for your playful room,
To Sony Playstation, taking you to another planet's moon.
Your world is full of wonders, just waiting to be unfurled,
With The Very Group, you'll discover your beautiful world!


## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [4]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [5]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [6]:
response = client.completions.create(
    model='gpt-35-turbo-instruct' , 
    prompt=prompt, 
    temperature=0,
    max_tokens=60
    )

print(response.choices[0].text)


- OpenAI and Microsoft are extending their partnership
- Microsoft is making a multi-year, multi-billion dollar investment in OpenAI
- This follows previous investments in 2019 and 2021
- The partnership will allow OpenAI to continue independent research and develop safe, useful, and powerful


## 4. Articulate the desired output format through examples

In [7]:
system_message = "You are a helpful assistant."

In [8]:
user_message=f"""Extract the entities mentioned in the text below. 
First extract all company names, then extract all years, 
then extract specific topics which fit the content and finally extract general overarching themes\n\n 
Desired format: 
Company names: <comma_separated_list_of_company_names> 
Years: 
Specific topics:
General themes: 
### Text:
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n\n 
###
"""


In [9]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft
Years: 2019, 2021
Specific topics: Partnership extension, Multi-year investment, Independent research, AI development
General themes: Corporate partnership, Investment in technology, AI research and development


## 5.Start with zero-shot, then few-shot (example)

In [10]:
system_message = "You are a helpful assistant."

In [11]:
prompt_zero=f"""Extract most important keywords from the corresponding texts below.\n\n 

###Text: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords:###
"""

In [12]:
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_zero}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, extending partnership, multi-year, multi-billion dollar investment, previous investments, 2019, 2021, independent research, develop AI, safe, useful, powerful.


In [13]:
prompt_few=f"""Extract most important keywords from the corresponding texts below.\n\n 
### Text 1: 
Stripe provides APIs that web developers can use to integrate 
payment processing into their websites and mobile applications. \n
Keywords 1: Stripe, payment processing, APIs, web developers, websites 
### 

###Text 2: 
OpenAI has trained cutting-edge language models that are very good at understanding 
and generating text. Our API provides access to these models and can be used to solve virtually 
any task that involves processing language. \n
Keywords 2: OpenAI, language models, text processing, API.
### 

###Text 3: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords 3:"""

In [14]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_few}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


## 6.Instead of just saying what not to do, say what to do instead

In [15]:
system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. 
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [16]:
response = client.chat.completions.create(
    model="gpt-4-32k", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

I'm sorry to hear that you're having trouble logging in. This can often be due to incorrect username or password. Please ensure there are no typographical errors that might prevent logging in. If this doesn't work, you might want to consider resetting your password. You can do this by visiting our Help guide at www.samplewebsite.com/help/faq where you can find step-by-step instructions to assist in resetting your password.


## 7. Divide complex tasks into sub-tasks

In [17]:
system_message = "You are a helpful assistant."

In [22]:
text = f"""
The Very Group: As the way families shop has changed, so have we. From catalogues, to bricks, to clicks, to mobile. We have consistently transformed to meet their needs.
Our ambition is to build the most trusted ecosystem for families. We want to offer the most flexible ways to pay for all the brands, products, and services they want and need within a seamless digital experience. To achieve our ambition, we are focused on three things: ease, choice and understanding.
"""
# example 1
user_message = f"""
Perform the actions below by separating your answers with line breaks. 
1 - Summarize the following text below with 1 sentence.
2 - Translate the summary into Turkish.
3 - List each company name in the Turkish summary.
4 - Output a json object that contains the following:
keys: turkish_summary, turkish_company_names.

###
Text:
{text} 
###
"""

In [23]:
response = client.chat.completions.create(
    model="gpt-4-32k", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

1 - The Very Group continuously adapts to changing consumer patterns, aiming to build the most trusted ecosystem by offering flexible payment methods and diverse brands through a seamless digital experience.
2 - The Very Group, tüketici alışkanlıklarındaki değişikliklere sürekli adapte olurken, en güvenilir ekosistemi kurmayı hedefliyor ve kusursuz bir dijital deneyim içinde çeşitli markalar için esnek ödeme yöntemleri sunmayı hedefliyor.
3 - The Very Group
4 - {
        "turkish_summary": "The Very Group, tüketici alışkanlıklarındaki değişikliklere sürekli adapte olurken, en güvenilir ekosistemi kurmayı hedefliyor ve kusursuz bir dijital deneyim içinde çeşitli markalar için esnek ödeme yöntemleri sunmayı hedefliyor.",
        "turkish_company_names": "The Very Group"
      }


## 8. Chain of Thought

The language model is prompted to generate a few intermediate reasoning steps to arrive at the final answer. 

Uses "greedy decoding" which means selecting the most likely token (word or character) at each step of the sequence generation process. At each time step, the model predicts the next token based on the previously generated tokens, and the token with the highest predicted probability is chosen as the output for that step. This process is repeated until the desired sequence length is reached or until a special end-of-sequence token is generated.

**Temp=0** is used because it uses greedy decoding. It first creates the greedy coding and then the answer.

In [24]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [25]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_ZERO_SHOT, 
    temperature=0,
    max_tokens=60
    )

print(response.choices[0].text)


6


In [5]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [6]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_ZERO_SHOT_CoT, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

1. Roger has 5 tennis balls.
2. He buys 2 cans of tennis balls, each with 3 balls. So, he has 5 + 2 x 3 = 11 tennis balls.
3. He gives 4 of them to his friend. So, he now has 11 - 4 = 7 tennis balls.
Therefore, Roger now has 7 tennis balls.


In [7]:
PROMPT_FEW_SHOT_CoT = """
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A: Elif had £10 at the beginning. When she consumed £2, 10-2=8 , £8 remains.
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A:
"""

In [8]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_FEW_SHOT_CoT, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.


** Auto-COT ** uses zero-shot-cot results just like few-shot learning for reasoning. Instead of using few-shot-cot, auto-cot can be useful and easy because you don't need to create manual examples (labels/reasonings)

In [16]:
prompt_auto_cot = """
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Lets think step by step.Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A:
"""

In [17]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=prompt_auto_cot, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)


Elif now has £8 left.


## 9. Self Consistency

Self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to ** select the most consistent answer.**

In the chat scenarios, **Asking the model to self-verify** its own responses. Like a student double-checking their answers, the AI model cross-references its responses to maintain consistency. 

In [22]:
prompt= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister?"""

In [24]:
response = client.completions.create(
    model="gpt-35-turbo-yk", 
    prompt=prompt, 
    temperature=0,
    max_tokens=10
    )

print(response.choices[0].text)



Q: 35
A: When I


In [21]:
prompt2= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister? Let's think step by step"""

In [26]:
response = client.completions.create(
    model="gpt-35-turbo-yk", 
    prompt=prompt2, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

.

When I was 6, my sister was half my age. So, my sister was 6/2=<<6/2=3>>3 years old.
Now I am 70. So, my sister is 70-6+3=<<70-6+3=67>>67 years old. Answer: \boxed{67}.<|im_end|>


In [37]:
prompt3=f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

In [38]:
def call_openai(num_times, start_phrase, temperature):
    for i in range(num_times):
        
        deployment_name='gpt-35-turbo-instruct' 

        # Send a completion call to generate an answer
        response = client.completions.create(
            model=deployment_name, 
            prompt=start_phrase, 
            temperature=temperature,
            max_tokens=200)
        print(response.choices[0].text)
        print("*****************************")

In [39]:
call_openai(10, prompt3, temperature = 1)

 When you were 6, your sister was half your age, so she was 6 / 2 = 3 years old. Now you are 70 and
your sister is 70 - (6 - 3) = 67 years old. The answer is 67.
*****************************
 When you were 6, your sister was half your age which would be 3 years old. From this, we can infer that your sister is 3 * 12 = 36 years younger than you. Therefore, your sister must be 70 - 36 = 34 years old now. The answer is 34.
*****************************
 Your sister is still half your age. When you were 6, she was 3 (half of 6). You are 64 years older than
she is. So now your sister is 70 - 64 = 6 years old.
*****************************
 Since you were 6, your sister was half your age, so she must have been 6 / 2 = 3 years old. The
difference in age always remains the same, so now when you are 70, your sister must be 70 - 3 = 67
years old. The answer is 67.
*****************************
 If you are 70 and your sister was half your age when you were 6, that means she was 70 / 2 = 35 years

## 10. Iterative approach

Prompt engineering is an iterative process. If you're unsatisfied with the AI's response, refine your prompt and try again. Analyze the results you receive and consider adjusting your prompt's context, clarity, or structure. This process of trial and error will help you better understand how the AI model interprets your prompts and allow you to fine-tune your approach.

·        Try different prompts to find what works best

·        When attempting few-shot learning, try also to include direct instructions

·        Rephrase a direct instruction set to be more or less concise, e.g.: taking a previous example and giving the next instruction without having to repeat the input

·        Try different personas keywords to see how it affects the response style

·        Use fewer or more examples in the few-shot learning

·        Co-create with AI: An example of a very useful prompt to get a good output from the LLM :

In [40]:
text=f"""
Location Action and impact
Global We updated and expanded our audit policy to require annual audits from all suppliers including both tier 1 and priority tier 2 factories. We defined remediation for different gradings from 
low to high risk and developed an escalation process that is aligned across all regions.
China More than 800 workers took part in our women’s empowerment programme, delivered in partnership with INNO. Modules are designed to boost skills and have a positive impact both in the 
workplace and personally.
China 100 participants from Tier 1 factories and 25 participants from Tier 2 factories took part in online training about ethical recruitment. Topics included raising awareness of and preventing 
modern slavery and child labour.
India Continued our UN-award winning work in India’s textile industry. To date, we have opened six migrant resource centres and will open a seventh centre during FY24. In FY23, 13 female 
migrant workers were supported with retrieving unpaid wages and returning home, in line with their wishes.
Malaysia
4 factories began reimbursing fees paid by workers during the recruitment process. Such fees are not permitted, as they are linked to modern slavery. 
5 factories began progressing plans to implement responsible recruitment. Sometimes referred to as ethical or fair recruitment, this means treating workers lawfully and in a fair and 
transparent manner that respects and protects their rights throughout recruitment, employment, and post-termination. 
Poland and Romania The Just Good Work app we developed and launched in FY22 was used by 1,532 Ukrainian refugees in Poland and Romania in FY23. The app provides labour rights and human rights 
guidance to prevent refugee exploitation.
Sri Lanka In response to the economic crisis in Sri Lanka, risk assessments were carried out in two factories to help us better understand conditions for workers in our supply chain. No negative 
responses were received from the 776 workers we engaged with.
Turkey Social dialogue training was conducted in three factories, with 1,018 participants. Delivered in partnership with Varner, Tesco, Primark, M&S, and JustMaxIt, the training was designed to 
strengthen communication between workers and managers.
Leicester, UK Provided continued support for FAB-L’s community outreach programme for garment workers, of which we were founding members. In FY23, our support helped FAB-L to introduce English 
language and IT classes and hire two new employees to continue and expand their essential work.
UK Visited all of our UK furniture suppliers to carry out assessment of non-conformances from third party audits, and observations of modern slavery indicators. No cases of modern slavery were 
identified. We also worked with suppliers on follow up actions and supported them with compliance to our audit programme.
UK 50% of our UK furniture suppliers attended a modern slavery training event we held in partnership with partnership with Slave-Free Alliance.
UK We began to consider our broader ESG commitments through the lens of preventing modern slavery by carrying out due diligence on our UK-based third-party take-back scheme partners. 
This included site visits to Traid, Reskinned and Vision Ireland
"""

In [41]:
prompt_iterative= f""" Your task is to explain given information in a very simple way.
### Context:
{text}
"""

In [42]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative}
    ]
)

print(response.choices[0].message.content)

In simpler terms:

Around the world, we have strengthened our policy to check our suppliers more regularly and ensure they are operating responsibly. We have special steps in place if we find any risky behavior.

In China, we have programs aimed at empowering women, with over 800 participants. We also provided online training about responsible recruitment to 125 people from our supplier factories.

In India, we support textile workers, especially migrant women. We have created places for them to go for resources, and helped some recover unpaid wages. 

In Malaysia, four of our factories have started reimbursing recruitment fees to workers, as these fees are tied to unfair practices. Five more factories are planning to implement fairer recruiting methods.

In Poland and Romania, we launched an app offering advice on labor and human rights to Ukrainian refugees. Over 1500 have used it so far.

In Sri Lanka, we assessed two of our factories to better understand the challenges of workers, 

**Issue 1:** I want to keep numerical values in more readable output format.

In [50]:
prompt_iterative1= f""" Your task is to organize given information in table format by keeping numarical values. 

### Context:
{text}
###
"""

In [51]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative1}
    ]
)

print(response.choices[0].message.content)

| Location | Numerical Information/Action |
| --- | --- |
| Global | Updated and expanded audit policy to require annual audits from all suppliers|
| China | 800 workers took part in women’s empowerment programme |
| China | 100 participants from Tier 1 factories and 25 participants from Tier 2 factories took part in online training about ethical recruitment |
| India | Opened six migrant resource centres, will open a seventh centre during FY24. Supported 13 female migrants in FY23 |
| Malaysia | 4 factories began reimbursing fees paid by workers, 5 factories began progressing plans for responsible recruitment |
| Poland and Romania | Just Good Work app used by 1,532 Ukrainian refugees |
| Sri Lanka | Risk assessments done in two factories, engaged with 776 workers |
| Turkey | Social dialogue training conducted in three factories with 1,018 participants |
| Leicester, UK | Supported FAB-L community outreach programme |
| UK | Visited all UK furniture suppliers for assessment and follo

**Issue 2:** It is long so I need a brief summary.

In [52]:
prompt_iterative2= f""" 
Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation by using at most 20 words.

### Context:
{text}
###
"""

In [53]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative2}
    ]
)

print(response.choices[0].message.content)

| Location       | Action                                                                                                             | Impact                                           |
|----------------|--------------------------------------------------------------------------------------------------------------------|-------------------------------------------------|
| Global         | Updated audit policy                                                                                                | Annual audits from all suppliers required       |
| China          | Women’s empowerment programme                                                                                       | Over 800 workers participated                   |
| China          | Online ethical recruitment training                                                                                 | 125 participants from Tier1 and Tier2 factories |
| India          | Continuing UN-award winning work in textile ind

In [59]:
prompt_iterative3= f""" Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation of the given context by using at most 20 words.

Format everything as HTML that can be used in a website. 

### Context:
{text}
###
"""

In [60]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative3}
    ]
)

print(response.choices[0].message.content)

```html
<table>
  <tr>
    <th>Location</th>
    <th>Action and Impact</th>
    <th>Numerical Values</th>
  </tr>
  <tr>
    <td>Global</td>
    <td>Updated and expanded audit policy including the development of an escalation process; defined remediation for gradings.</td>
    <td></td>
  </tr>
  <tr>
    <td>China</td>
    <td>Women’s empowerment program impacted over 800 workers.</td>
    <td>800</td>
  </tr>
  <tr>
    <td>China</td>
    <td>Conducted online training about ethical recruitment for Tier 1 and Tier 2 factories.</td>
    <td>100, 25</td>
  </tr>
  <tr>
    <td>India</td>
    <td>Opened migrant resource centres; supported female migrant workers with unpaid wages.</td>
    <td>6, 13</td>
  </tr>
  <tr>
    <td>Malaysia</td>
    <td>Began reimbursing recruitment fees and implementing responsible recruitment across factories.</td>
    <td>4, 5</td>
  </tr>
  <tr>
    <td>Poland and Romania</td>
    <td>Provided labor and human rights guidance via an app to Ukrainian refugee

In [61]:
from IPython.display import display, HTML
display(HTML(response.choices[0].message.content))